In [1]:
%reload_ext autoreload
%autoreload 2

In [2]:
import chess
from playing.utils import board_to_piece_uci_moves
from playing.agents import GPTEloAgent, NegaMaxMaterialGPTEloAgent
from training.lightning_training import LightningGPT

from data_process import tokenizers
from nanoGPT.model import GPTConfig
import json

In [3]:
from training import load_models


material_checkpoint = "./models/full_training/elo_material_ignore_material_prediction/epoch=9-step=1250000.ckpt"
material_model = load_models.material_model(material_checkpoint)

material_pair_checkpoint = "./models/full_training/elo_material_pair_ignore_material_prediction/epoch=9-step=1250000.ckpt"
material_pair_model = load_models.material_pair_model(material_pair_checkpoint)

piece_count_checkpoint = "./models/full_training/elo_piece_count_ignore_material_prediction/epoch=9-step=1250000.ckpt"
piece_count_model = load_models.piece_count_model(piece_count_checkpoint)

base_checkpoint = "./models/full_training/masked_elo/epoch=9-step=1250000.ckpt"
base_model = load_models.base_elo_model(base_checkpoint)

base_no_mask_checkpoint = "./models/full_training/adaptive_elo/epoch=9-step=1250000.ckpt"
base_no_mask_model = load_models.base_elo_no_mask_model(base_no_mask_checkpoint)

no_elo_checkpoint = "./models/full_training/no_elo/epoch=9-step=1250000.ckpt"
no_elo_model = load_models.no_elo_model(no_elo_checkpoint)

number of parameters: 27.34M
number of parameters: 28.15M
number of parameters: 28.17M
number of parameters: 27.29M
number of parameters: 27.29M
number of parameters: 27.27M


In [4]:
import pandas as pd
from data_process.utils import remove_last_player_material_token, remove_material_tokens


games_df = pd.read_csv("./data/test_piece_count.csv", delimiter=";")

# games_df = games_df.sample(frac=0.1, random_state=42)
games_df = games_df[(games_df["white_elo"] // 100 * 100 == 1500) & (games_df["black_elo"] // 100 * 100 == 1500)]

# games = remove_material_tokens(games_df.piece_uci)
# games = add_elo_token_to_games(games, games_df.white_elo, games_df.black_elo)

games_df

,index,id,date,white_elo,black_elo,result,ply,ply_30s,piece_uci
20006,6987904,wjpxZGxT,2024.12.03,1514,1523,0-1,32,-1,Pe2e4 Q1R2B2N2P8 Q1R2B2N2P8 Pd7d5 Q1R2B2N2P8 Q...
20011,6987938,drV1Uovt,2024.12.03,1559,1513,1-0,83,-1,Pe2e4 Q1R2B2N2P8 Q1R2B2N2P8 Pe7e5 Q1R2B2N2P8 Q...
20014,6987974,ldQxLyUL,2024.12.03,1547,1549,0-1,62,-1,Pe2e4 Q1R2B2N2P8 Q1R2B2N2P8 Pe7e6 Q1R2B2N2P8 Q...
20015,6987977,b4h6PnnZ,2024.12.03,1522,1535,1-0,79,61,Pd2d4 Q1R2B2N2P8 Q1R2B2N2P8 Pe7e6 Q1R2B2N2P8 Q...
20016,6987978,0CztK44T,2024.12.03,1582,1500,0-1,74,-1,Pe2e4 Q1R2B2N2P8 Q1R2B2N2P8 Pd7d5 Q1R2B2N2P8 Q...
...,...,...,...,...,...,...,...,...,...
29988,7071818,0zMw0TRU,2024.12.03,1523,1541,1-0,49,33,Pe2e4 Q1R2B2N2P8 Q1R2B2N2P8 Pe7e6 Q1R2B2N2P8 Q...
29989,7071833,uCPjZ7tj,2024.12.03,1524,1528,0-1,130,-1,Pe2e4 Q1R2B2N2P8 Q1R2B2N2P8 Pc7c6 Q1R2B2N2P8 Q...
29992,7071843,4OAj3ye9,2024.12.03,1549,1546,1-0,55,-1,Pe2e4 Q1R2B2N2P8 Q1R2B2N2P8 Pg7g6 Q1R2B2N2P8 Q...
29996,7071926,wDiTn8km,2024.12.03,1567,1583,0-1,26,-1,Pe2e4 Q1R2B2N2P8 Q1R2B2N2P8 Pe7e5 Q1R2B2N2P8 Q...


In [5]:
def elo_agent_prediction(agent, piece_uci, white_elo, black_elo):
    if isinstance(piece_uci, str):
        piece_uci = piece_uci.split(" ")
    if isinstance(white_elo, int):
        white_elo = str(white_elo)
    if isinstance(black_elo, int):
        black_elo = str(black_elo)

    board = chess.Board()

    correct_predictions = 0
    all_predictions = 0

    for move in piece_uci:
        if board.turn == chess.WHITE:
            agent.model_elo = white_elo
        else:
            agent.model_elo = black_elo

        predicted_move = agent.play(board)
        real_move = chess.Move.from_uci(move[1:])
        if predicted_move == real_move:
            correct_predictions += 1
        all_predictions += 1
        board.push(chess.Move.from_uci(move[1:]))

    return correct_predictions, all_predictions

In [ ]:
from tqdm import tqdm

def move_probabilities(agent, game, white_elo, black_elo):
    if isinstance(game, str):
        game = game.split(" ")
    if isinstance(white_elo, int):
        white_elo = str(white_elo)
    if isinstance(black_elo, int):
        black_elo = str(black_elo)

    board = chess.Board()


    moves_probabilities = []


    for i in range(0, len(game), 3):
        move = game[i]
        
        if board.turn == chess.WHITE:
            agent.model_elo = white_elo
        else:
            agent.model_elo = black_elo

        current_move_probabilities = agent.legal_moves_probabilities(board)

        moves_probabilities.append({
            "fen": board.fen(),
            "played_move": move,
            "probabilities": current_move_probabilities
        })

        board.push_uci(move[1:])

    return moves_probabilities

def process_game(agent, row):
    game_str = row.piece_uci.split(" ")
    game = game_str[::3]
    white_elo = (row.white_elo // 100 * 100)
    black_elo = (row.black_elo // 100 * 100)

    game_moves_probabilities = move_probabilities(agent, game_str, white_elo, black_elo)

    return {
        "game": game,
        "white_elo": row.white_elo,
        "black_elo": row.black_elo,
        "result": row.result,
        "game_moves_probabilities": game_moves_probabilities,
        
    }


def test_elo_agent(agent, games_df):
    results = []
    for row in tqdm(games_df.itertuples(index=False), total=len(games_df)):
        result = process_game(agent, row)
        results.append(result)
    return results

In [13]:
games_df_sample = games_df.sample(n=10, random_state=42)
agent = GPTEloAgent(model=piece_count_model.model, model_elo=1500, notes="piece_count")

test_elo_agent(agent, games_df_sample)

100%|██████████| 10/10 [00:23<00:00,  2.33s/it]


[{'game': ['Pd2d4',
   'Pd7d5',
   'Ng1f3',
   'Ng8f6',
   'Ph2h3',
   'Pc7c5',
   'Bc1f4',
   'Nb8c6',
   'Pc2c3',
   'Qd8b6',
   'Qd1b3',
   'Pc5c4',
   'Qb3b6',
   'Pa7b6',
   'Pe2e3',
   'Bc8f5',
   'Bf1e2',
   'Ph7h6',
   'Ke1g1',
   'Pe7e6',
   'Nf3e5',
   'Bf8e7',
   'Pa2a4',
   'Nc6e5',
   'Pd4e5',
   'Nf6e4',
   'Pf2f3',
   'Ne4c5',
   'Be2d1',
   'Ke8g8',
   'Pg2g4',
   'Bf5h7',
   'Rf1f2',
   'Pb6b5',
   'Pa4a5',
   'Pb7b6',
   'Nb1a3',
   'Ra8a5',
   'Pg4g5',
   'Be7g5',
   'Bf4g5',
   'Ph6g5',
   'Rf2g2',
   'Rf8a8',
   'Rg2g5',
   'Nc5d3',
   'Rg5g2',
   'Nd3e5',
   'Pf3f4',
   'Ne5d3',
   'Ph3h4',
   'Bh7e4',
   'Rg2h2',
   'Kg8f8',
   'Ph4h5',
   'Kf8e7',
   'Ph5h6',
   'Pg7h6',
   'Rh2h6',
   'Be4g6',
   'Rh6h2',
   'Nd3c5',
   'Bd1h5',
   'Bg6h5',
   'Rh2h5',
   'Ra8g8',
   'Kg1f2',
   'Nc5e4',
   'Kf2f1',
   'Ne4g3'],
  'white_elo': 1517,
  'black_elo': 1564,
  'result': '0-1',
  'game_moves_probabilities': [{'fen': 'rnbqkbnr/pppppppp/8/8/8/8/PPPPPPPP/RNBQKBNR w KQkq

In [7]:
with open("move_probabilities_2.json", "r") as f:
    move_probabilities_data = json.load(f)

In [8]:
move_probabilities_data[0]

{'game': ['Pe2e4',
  'Pd7d5',
  'Pe4e5',
  'Pd5d4',
  'Pf2f4',
  'Pa7a6',
  'Pd2d3',
  'Nb8c6',
  'Ng1f3',
  'Bc8f5',
  'Bf1e2',
  'Pe7e6',
  'Pc2c3',
  'Ph7h6',
  'Pa2a3',
  'Ng8e7',
  'Pc3d4',
  'Nc6d4',
  'Nf3d4',
  'Qd8d4',
  'Nb1c3',
  'Ke8c8',
  'Be2f3',
  'Qd4d3',
  'Qd1b3',
  'Ne7c6',
  'Ke1f2',
  'Bf8c5',
  'Kf2g3',
  'Ph6h5',
  'Rh1d1',
  'Ph5h4'],
 'white_elo': 1514,
 'black_elo': 1523,
 'result': '0-1',
 'game_moves_probabilities': [{'fen': 'rnbqkbnr/pppppppp/8/8/8/8/PPPPPPPP/RNBQKBNR w KQkq - 0 1',
   'played_move': 'Pe2e4',
   'probabilities': {'Pe2e4': 0.659928023815155,
    'Pd2d4': 0.22540520131587982,
    'Pc2c4': 0.026457708328962326,
    'Ng1f3': 0.02323821745812893,
    'Pe2e3': 0.013862695544958115,
    'Pb2b3': 0.011310497298836708,
    'Pg2g3': 0.009280737489461899,
    'Pf2f4': 0.008340497501194477,
    'Pd2d3': 0.005025583319365978,
    'Pb2b4': 0.0040827286429703236,
    'Nb1c3': 0.003880030708387494,
    'Pg2g4': 0.003311620792374015,
    'Pc2c3': 0.00198571

In [9]:
with open("move_probabilities_formatted.json", "w") as f:
    json.dump(move_probabilities_data, f, indent=2)